In [1]:
from gensim.models.doc2vec import  Doc2Vec, TaggedDocument
import pandas as pd
from sklearn.cluster import KMeans
import random
import statsmodels.api as sm
from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import chi2_contingency
from nltk.stem.snowball import SnowballStemmer
import re
import nltk
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from NLP.supervized_clustering_metrics import precison_recall_f1
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
#import Stemmer
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import json
from multiprocessing import Pool
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

C:\Users\Кирилл\AppData\Local\Programs\Python\Python36-32\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Имеем файл data.json - это список всех имен товаров, предварительно очищеный от стоп слов и лемматизированный.

Имеем файл test_task.json - это сгруппированые товары - всего 1223 группы. Как проводилась группировка неизвестно, возможно это делали люди, возможно какой либо алгоритм.

Задача: предложить лучший способ кластеризации имен товаров.

Для кластеризации имен товаров будем использовать tf-idf векторизацию. Далее матрицу этих векторов отправим в spherical k-means для кластеризации. Количество кластеров задаем 1223 как в файле с образцом группировки. Spherical k-means - потому что он использует как меру близости векторов cousine similarity, это native метрика для близости векторов. Обычный k-means использует евклидово расстояние. Использование spherical k-means прибавляет к качеству кластеризации в данном случае 0.01%. 


In [ ]:
with open('C:\\Users\\Кирилл\\PycharmProjects\\BI\\NLP\\data.json') as f:
    data = json.load(f)
prep_docs = json.loads(data)

dictionary = {}

documents2 = []
for doc in prep_docs:
    doc2 = doc.lower()
    tokens = [word for word in nltk.word_tokenize(doc2)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('/', token):
           tt = token.split("/")
           tot = len(tt)
           kk=0
           for k in range(0,tot,1):
               if (len(tt[k-kk]) <= 1):
                    del tt[k-kk]
                    kk += 1
           token = " ".join(tt)
           #filtered_tokens.append("".join(tt))

        if len(token)>1:
            filtered_tokens.append(token)
            if token not in dictionary:
                dictionary[token] = 1
            else:
                dictionary[token] += 1
    # stems = [stemmer.stem(t) for t in filtered_tokens]
    # stems2 = [stemmer2.stem(t) for t in stems]

    sentence = ' '.join(filtered_tokens)
    documents2.append(sentence)


prep_docs = documents2
documents2 = []
for doc in prep_docs:
    doc2 = doc.lower()
    tokens = [word for word in nltk.word_tokenize(doc2)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:

        if token not in dictionary \
                or token=="зл" or token=="сие" or token=="лен" or token == "упак" or token == "ав" or token == "ава":
            continue
        if dictionary[token]==1:
            continue

        filtered_tokens.append(token)

    # stems = [stemmer.stem(t) for t in filtered_tokens]
    # stems2 = [stemmer2.stem(t) for t in stems]

    sentence = ' '.join(filtered_tokens)
    documents2.append(sentence)

prep_docs = documents2

 Убрали мусорные токены - обрывки слов. Разделили слова соединенные через слеш. Убрали слова с частотой в корпусе равной единице. 


In [7]:
df = pd.read_json("C:\\Users\\Кирилл\\PycharmProjects\\BI\\NLP\\test_task.json",orient='columns', encoding='utf-8')

clusters = df.shape[0]
max_group_size = df.shape[1]

sentenceLabeled = []
#df[:1]

before = {}
before_sent = {}

documents = []

sentenceID = 0
b = np.array([])
for group in range(0,clusters,1):
    for col in range(0, max_group_size, 1):
        if (df[col][group] != None):
            #sentenceL = TaggedDocument(words=df[col][group].lower().split(), tags=['SENT_%s' % sentenceID])
            # sentenceL = TaggedDocument(words=sentence.split(), tags = ['SENT_%s' %sentenceID])
            #sentenceLabeled.append(sentenceL)
            #before['SENT_%s' % sentenceID] = group
            #before_sent['SENT_%s' % sentenceID] = df[col][group]
            #documents.append(df[col][group])
            b = np.append(b, int(group))
            sentenceID += 1




переменная b - это массив в котором значения - принадлежность товара к группе в данном нам файле, значения массива - номер группы к которой принадлежит этот товар

In [8]:
tfidf_vectorizer = TfidfVectorizer(analyzer="word", max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(prep_docs)

In [9]:
def select_n_components(var_ratio, goal_var: float):
    # Set initial variance explained so far
    total_variance = 0.0

    # Set initial number of features
    n_components = 0

    # For the explained variance of each feature:
    for explained_variance in var_ratio:

        # Add the explained variance to the total
        total_variance += explained_variance

        # Add one to the number of components
        n_components += 1

        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break

    # Return the number of components
    return n_components

svd = TruncatedSVD(tfidf_matrix.shape[1]-1)
svd.fit(tfidf_matrix)

tsvd_var_ratios = svd.explained_variance_ratio_
n = select_n_components(tsvd_var_ratios, 0.98)
svd = TruncatedSVD(n)

#todo добавить функцию делающую количество компонент связности чтобы потом вставить её в TruncatedSVD
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(tfidf_matrix)


Манипуляции выше нужны чтобы сделать sperical k-means

In [10]:
km = KMeans(n_clusters=clusters)
km.fit(X)
clusters = km.labels_.tolist()

Вычисляем метрики для кластеризации как таковой. А так же метрики с учетом true классов: precison, recall, f1. Для этого написана одноименная функция по мотивам https://stats.stackexchange.com/questions/15158/precision-and-recall-for-clustering?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa


In [12]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(b, clusters))
print("Completeness: %0.3f" % metrics.completeness_score(b, clusters))
print("V-measure: %0.3f" % metrics.v_measure_score(b, clusters))
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(b, clusters))

indeces_of_groups = {}
dict_list = []
for i in range(0,len(clusters),1):
    val = clusters[i]
    if val not in indeces_of_groups:
        indeces_of_groups[val] = []
        indeces_of_groups[val].append(str(b[i]))
    else:
        indeces_of_groups[val].append(str(b[i]))

[dict_list.append(v) for k,v in indeces_of_groups.items()]

precison_recall_f1(dict_list)



Homogeneity: 0.935
Completeness: 0.939
V-measure: 0.937
Adjusted Rand-Index: 0.532


num_doc is 3580 
positives is 4938 
TP is 2528 
FP is 2410 
FN is 2030 
TN is 6399442 
Precision is 0.51 
Recall is 0.55 
F1 is 0.53 
